In [2]:
import angr

proj = angr.Project('./binaries/game.exe',  auto_load_libs=False)

cfg = proj.analyses.CFG()

proj.kb.functions.values()

WARNING  | 2024-09-11 10:29:52,137 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Binop Iop_InterleaveLO8x16.


ValuesView(<angr.knowledge_plugins.functions.function_manager.FunctionManager object at 0x0000010D96267380>)

In [4]:
for func in proj.kb.functions.values():
    print(f"Function {func.name} at address {func.addr}")
    for block in func.blocks:
        print(f"  Basic block at {block.addr} with instructions:")
        block.pp()  # pretty print the instructions in this block

Function UnityMain at address 5368713216
  Basic block at 5368713216 with instructions:
           UnityMain:
140001000  jmp     qword ptr [0x14000c210]
Function sub_140001008 at address 5368713224
  Basic block at 5368713224 with instructions:
140001008  push    rbx
14000100a  sub     rsp, 0x20
14000100e  mov     ecx, 0x2
140001013  call    sub_140002b04
  Basic block at 5368713240 with instructions:
140001018  call    sub_14000157c
  Basic block at 5368713245 with instructions:
14000101d  mov     ecx, eax
14000101f  call    sub_14000352c
  Basic block at 5368713252 with instructions:
140001024  call    sub_140001570
  Basic block at 5368713257 with instructions:
140001029  mov     ebx, eax
14000102b  call    sub_1400036b8
  Basic block at 5368713264 with instructions:
140001030  mov     ecx, 0x1
140001035  mov     dword ptr [rax], ebx
140001037  call    sub_1400012fc
  Basic block at 5368713276 with instructions:
14000103c  test    al, al
14000103e  je      0x1400010b3
  Basic block 

In [4]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

node_text = "Main API call or function name"
embedding = model.encode(node_text)


edge_index = torch.tensor([[0], [0]], dtype=torch.long)
data = Data(x=torch.tensor([embedding], dtype=torch.float), edge_index=edge_index)



class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            torch.nn.Sequential(
                torch.nn.Linear(384, 64),  # Input: 384-dim embeddings, Output: 64-dim
                torch.nn.ReLU(),
                torch.nn.Linear(64, 64)
            )
        )
        self.conv2 = GINConv(
            torch.nn.Sequential(
                torch.nn.Linear(64, 64),
                torch.nn.ReLU(),
                torch.nn.Linear(64, 64)
            )
        )
        self.fc1 = torch.nn.Linear(64, 32)
        self.fc2 = torch.nn.Linear(32, 2)  # Binary classification: malware or benign

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        
        # Global pooling (summing node embeddings to create a graph embedding)
        x = global_add_pool(x, torch.zeros(x.size(0), dtype=torch.long))
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Initialize the GIN model
model = GIN()

# Example forward pass with the constructed graph
output = model(data)

tensor([[-0.6167, -0.7759]], grad_fn=<LogSoftmaxBackward0>)


In [6]:
probabilities = torch.exp(output)
print(probabilities)
predicted_class = torch.argmax(output, dim=1)
print(f"Predicted class: {predicted_class.item()}")

tensor([[0.5397, 0.4603]], grad_fn=<ExpBackward0>)
Predicted class: 0
